In [ ]:
Чтение файла и первичное знакомство с содержимым

In [ ]:
import pandas as pd
df = pd.read_excel("Dataset 1.xlsx")
print("Названия столбцов:")
print(df.columns)

print("\nУникальные значения в 'Тип парка':")
print(df['Тип парка'].unique())

df_local = df[df['Тип парка'].str.contains("Местный", case=False, na=False)].copy()

print(f"\nРазмер данных с типом парка 'Местный': {df_local.shape}")
print("\nПервые строки:")
print(df_local.head())


In [ ]:
Разветочный анализ (EDA)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

print("Информация о целевой переменной 'Срок доставки факт':")
print(df_local['Срок доставки факт'].describe())
print("\nКоличество пропусков:", df_local['Срок доставки факт'].isna().sum())

plt.figure(figsize=(10, 5))
sns.histplot(df_local['Срок доставки факт'].dropna(), bins=50, kde=True)
plt.title('Распределение целевой переменной «Срок доставки факт»')
plt.xlabel('Срок доставки факт')
plt.ylabel('Количество записей')
plt.grid()
plt.tight_layout()
plt.show()

In [ ]:
Обзор типа данных и проверка данных на пустые значения

In [ ]:
df_local.info()

mis_data = df_local.isnull().sum()
mis_data = mis_data[mis_data > 0].sort_values(ascending=False)

mis_perc = (mis_data / len(df_local)).sort_values(ascending=False)

mis_sum = pd.DataFrame({
    'Количество пропусков': mis_data,
    'Доля пропусков (%)': (mis_perc * 100).round(2)
})

print("\nСтолбцы с пропущенными значениями:")
print(mis_sum.head(20))  


In [ ]:
Удаление признаков с отсутсвующими признаками и строк без целевой переменной

In [ ]:
df_model = df_local.dropna(axis=1, how='all')

df_model = df_model[df_model['Срок доставки факт'].notna()].copy()

print(f"Размер набора после очистки: {df_model.shape}")


In [ ]:
Просмотр признаков для анализа:

In [ ]:
print("Первые 30 признаков:")
print(df_model.columns[:30].tolist())


In [ ]:
Анализ признаков, влияющих на целевую переменную, на основе матрицы корреляции

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
numeric_features = [
    'Вес груза (кг)', 
    'Пробег в груженом состоянии (км)', 
    'Пробег в порожнем состоянии (км)', 
    'Пробег общий (км)', 
    'Норматив величины пробега (км)', 
    'Остаток пробега (км)', 
    'Нормативный срок доставки', 
    'Расстояние пройденное (км)', 
    'Простой', 
    'Срок доставки факт'
]

df_model.columns = df_model.columns.str.strip()

plt.figure(figsize=(12, 8))
corr_matrix = df_model[numeric_features].corr()
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap="coolwarm", square=True)
plt.title("Корреляционная матрица признаков и целевой переменной")
plt.tight_layout()
plt.show()


SyntaxError: invalid syntax (2327352157.py, line 2)

In [ ]:
Генерация признаков

In [ ]:
import pandas as pd
import numpy as np

df_feat = df_model.copy()

df_feat['Время_в_пути (ч)'] = (df_feat['Дата и время окончания рейса'] - df_feat['Дата и время начала рейса']).dt.total_seconds() / 3600
df_feat['Дата_начала_год'] = df_feat['Дата и время начала рейса'].dt.year
df_feat['Дата_начала_месяц'] = df_feat['Дата и время начала рейса'].dt.month
df_feat['Дата_начала_день_недели'] = df_feat['Дата и время начала рейса'].dt.dayofweek

cat = [
    'Род вагона', 
    'Наименование груза', 
    'Станция отправления', 
    'Станция назначения', 
    'Оператор'
]

for col in cat:
    df_feat[col] = df_feat[col].astype(str).str.strip()

num = [
    'Вес груза (кг)',
    'Пробег общий (км)',
    'Норматив величины пробега (км)',
    'Нормативный срок доставки',
    'Расстояние пройденное (км)',
    'Простой',
    'Время_в_пути (ч)'
]


target = 'Срок доставки факт'


df_final = df_feat[catl + num + [
    'Дата_начала_год', 'Дата_начала_месяц', 'Дата_начала_день_недели',
    target
]].dropna()

print("Форма витрины данных:", df_final.shape)
print("Признаки для моделирования:")
print(df_final.columns.tolist())


In [ ]:
Разделение на обучающую и тестовые выборки

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

X = df_final.drop(columns=['Срок доставки факт'])
y = df_final['Срок доставки факт']

X_encoded = pd.get_dummies(X, drop_first=True)

X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.2, random_state=42
)

print("Размер обучающей выборки:", X_train.shape)
print("Размер тестовой выборки:", X_test.shape)


In [ ]:
Построение модели с помощью AutoML (baseline)

In [ ]:
from lazypredict.Supervised import LazyRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import time

start_time = time.time()

model = LazyRegressor(verbose=1, ignore_warnings=True, custom_metric=None)
models, predictions = model.fit(X_train, X_test, y_train, y_test)

print(f"\nВремя выполнения AutoML baseline: {round(time.time() - start_time, 2)} сек.")
print("\nРезультаты:")
print(models)


In [ ]:
Ручное моделирование (RandomForest, XGBoost)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import LabelEncoder

def clean_column_names(df):
    df.columns = [str(col).replace('(', '').replace(')', '').replace(' ', '_').replace('-', '_').replace('/', '_')
                 for col in df.columns]
    return df

df = pd.read_excel("Dataset 1.xlsx")

df = df[df["Тип парка"].str.contains("Местный", na=False)]


date_cols = df.select_dtypes(include=['datetime64', 'datetime64[ns]']).columns
df = df.drop(columns=date_cols)

categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    df[col] = LabelEncoder().fit_transform(df[col].astype(str))

X = df.drop("Срок доставки факт", axis=1)
y = df["Срок доставки факт"]

X = clean_column_names(X)

X = X.fillna(X.median())
y = y.fillna(y.median())

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

results = {}

rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

results['RandomForest'] = {
    'R2': r2_score(y_test, y_pred_rf),
    'RMSE': np.sqrt(mean_squared_error(y_test, y_pred_rf)),
    'MAE': mean_absolute_error(y_test, y_pred_rf)
}

xgb = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=6, random_state=42)
xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict(X_test)

results['XGBoost'] = {
    'R2': r2_score(y_test, y_pred_xgb),
    'RMSE': np.sqrt(mean_squared_error(y_test, y_pred_xgb)),
    'MAE': mean_absolute_error(y_test, y_pred_xgb)
}


for model_name, metrics in results.items():
    print(f"\n{model_name}:")
    for metric_name, value in metrics.items():
        print(f"  {metric_name}: {value:.4f}")


In [ ]:
Получение интерпретации XGBoost

In [ ]:
importances = xgb.feature_importances_
feature_names = X_test.columns

sorted_idx = importances.argsort()[::-1][:20]
plt.figure(figsize=(10, 6))
plt.barh(range(len(sorted_idx)), importances[sorted_idx])
plt.yticks(range(len(sorted_idx)), feature_names[sorted_idx])
plt.gca().invert_yaxis()
plt.title("Топ-20 наиболее важных признаков (XGBoost)")
plt.xlabel("Важность")
plt.tight_layout()
plt.show()


In [ ]:
Локальная интерпретация с помощью Lime

In [ ]:
import lime
import lime.lime_tabular

samples = X_test.iloc[[0, 1, 2]]
sample_array = samples.values

explainer = lime.lime_tabular.LimeTabularExplainer(
    training_data=X_train.values,
    feature_names=X_train.columns.tolist(),
    mode='regression'
)

for i in range(3):
    print(f"\nИнтерпретация наблюдения {i}")
    explanation = explainer.explain_instance(
        data_row=sample_array[i],
        predict_fn=xgb.predict,
        num_features=10
    )
    explanation.save_to_file(f'lime_explanation_{i}.html')  

